<a href="https://colab.research.google.com/github/kiarashrahmani/CTR-Prediction/blob/main/CTR_GT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Load Data**

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/train.csv')

In [4]:
df.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,3.190000e+18,0,14102101,1005,0,85f751fd,c4e18dd6,50e219e0,c0c5eb49,f2f777fb,...,1,0,21611,320,50,2480,3,297,100111,61
1,3.190000e+18,1,14102101,1005,0,5b08c53b,7687a86e,3e814130,ecad2386,7801e8d9,...,1,0,17653,300,250,1994,2,39,-1,33
2,3.190000e+18,0,14102101,1005,0,5b08c53b,7687a86e,3e814130,ecad2386,7801e8d9,...,1,0,17653,300,250,1994,2,39,-1,33
3,3.190000e+18,0,14102101,1005,0,85f751fd,c4e18dd6,50e219e0,5e3f096f,2347f47a,...,1,0,21611,320,50,2480,3,297,100111,61
4,3.190000e+18,0,14102101,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15706,320,50,1722,0,35,100084,79


In [5]:
df.set_index('id',inplace=True)

In [6]:
y = df['click']

In [7]:
x = df.loc[:,df.columns!='click']

In [8]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 100001 entries, 3.19e+18 to 1.35e+19
Data columns (total 22 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   hour              100001 non-null  int64 
 1   C1                100001 non-null  int64 
 2   banner_pos        100001 non-null  int64 
 3   site_id           100001 non-null  object
 4   site_domain       100001 non-null  object
 5   site_category     100001 non-null  object
 6   app_id            100001 non-null  object
 7   app_domain        100001 non-null  object
 8   app_category      100001 non-null  object
 9   device_id         100001 non-null  object
 10  device_ip         100001 non-null  object
 11  device_model      100001 non-null  object
 12  device_type       100001 non-null  int64 
 13  device_conn_type  100001 non-null  int64 
 14  C14               100001 non-null  int64 
 15  C15               100001 non-null  int64 
 16  C16               100001 no

**Transformation**

In [24]:
from sklearn.preprocessing import LabelEncoder
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_columns = x.select_dtypes(include=numerics)
non_numeric_columns = list(set(x.columns) - set(numeric_columns)) + list(
    set(numeric_columns) - set(x.columns))
le = LabelEncoder()
x[non_numeric_columns] = x[non_numeric_columns].apply(le.fit_transform)

In [10]:
x.head()

,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
id,,,,,,,,,,,,,,,,,,,,,
3.190000e+18,14102101,1005,0,572,699,4,682,66,2,7607,...,1,0,21611,320,50,2480,3,297,100111,61
3.190000e+18,14102101,1005,0,398,409,3,853,30,0,7607,...,1,0,17653,300,250,1994,2,39,-1,33
3.190000e+18,14102101,1005,0,398,409,3,853,30,0,7607,...,1,0,17653,300,250,1994,2,39,-1,33
3.190000e+18,14102101,1005,0,572,699,4,374,8,2,7607,...,1,0,21611,320,50,2480,3,297,100111,61
3.190000e+18,14102101,1005,0,126,856,1,853,30,0,7607,...,1,0,15706,320,50,1722,0,35,100084,79


**Model Training**

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score


# Define the parameter grid to search
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_model = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, scoring='accuracy', cv=3)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_


In [13]:
y_predict = best_model.predict(X_test)

**Evaluation**

In [14]:
from sklearn.metrics import accuracy_score, precision_score
print(accuracy_score(y_pred=y_predict,y_true=y_test))
print(precision_score(y_pred=y_predict,y_true=y_test))

0.8394897124329566
0.5422661870503597


**Create final file**

In [15]:
df_test = pd.read_csv('/content/test.csv')

In [16]:
df_test.set_index('id',inplace=True)

In [17]:
from sklearn.preprocessing import LabelEncoder
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_columns = df_test.select_dtypes(include=numerics)
non_numeric_columns = list(set(df_test.columns) - set(numeric_columns)) + list(
    set(numeric_columns) - set(df_test.columns))
df_test[non_numeric_columns] = df_test[non_numeric_columns].apply(le.fit_transform)

In [19]:
y_predict = best_model.predict(df_test)

In [20]:
y_predict

array([0, 0, 0, ..., 0, 0, 0])

In [21]:
df_final= pd.DataFrame()

In [22]:
df_final['id'] = df_test.index
df_final['Click'] = y_predict

In [23]:
df_final.to_csv('submission.csv')